In [3]:
import pandas as pd
import json
import urllib
import re
import datetime
import pytz
import matplotlib.pyplot as plt
import io

# 01 load data(for raw data and clean data respectively)

## 1.1 load the raw data

In [4]:
#load the rae data and turn it into the json format(notes: you should put the data and the ipynb folder in a folder together)
from io import open
filepath='insertYourFilePath'
with open (filepath+'./2018-05-24_01 2','r', encoding='utf-8',errors='replace') as myfile:
    data = myfile.readlines()
    dataRawJson=[json.loads(data[i]) for i in range(len(data))]

IOError: [Errno 2] No such file or directory: './2018-05-24_01 2'

# 1.2 load the clean data into the json format and extract the valid macs in order to compare it with those in the raw data

In [5]:
with open ('./pri-2018-05-24_01','r',encoding='utf-8',errors='replace') as myfile:
    data = myfile.readlines() #load the clean data
    
dataFormatted = '['+re.sub('}\s*{','},{',data[0])+']' #methods for formatting the data: search for }{, ignoring whitespace, add a comma 
dataCleanJson=json.loads(dataFormatted)#format the clean data
dataCleanJsonLists=[[rec['data'][i].split(',')for i in range(len(rec['data']))]for rec in dataCleanJson]#split each row with comma
validMacs = [[record[i][5] for i in range(len(record))] for record in dataCleanJsonLists]#extract the valid macs in the clean data
uniqueValidMacs=set([mac for macList in validMacs for mac in macList])#get the unique valid macs
numUniqueValidMacs = len(uniqueValidMacs)#calculate the numbers of the valid macs

IOError: [Errno 2] No such file or directory: './pri-2018-05-24_01'

# 02 parse the raw data

## 2.1 select the valid macs and extract the useful features like probe/ts/datetime/rssi

In [ ]:
#set the basic encoding environment for elements in raw data,
timeCorrection=8*60*60
allTimes=[]
individuals={}
c=0
for record in dataRawJson:
    c+=1
    #print(c)
    wifi=record['wifidata']
    apmac=wifi['apmac']
    ts=float(wifi['tssend'])+timeCorrection #we should use the time when it send out the data which is more accurate
    allTimes.extend([ts])
    dateTime=datetime.datetime.utcfromtimestamp(ts/1000).strftime('%Y%m%d %H:%M:%S') #use datetime to transfer into the local time
    count=wifi['count']
    for obs in wifi['wifitalist']: #define information in wifitalist and
        valid=obs['mac'] in uniqueValidMacs
        newObs={'probe':apmac,
                'ts':ts,
                'datetime':dateTime,
                'rssi':obs['rssi']} #rename all the features in wifitalist
        if obs['mac'] in individuals:
            individuals[obs['mac']]['observations'].append(newObs)
        else:
            individuals[obs['mac']]={'valid':valid,'observations':[newObs]}
obsPerPerson = [len(individuals[i]) for i in individuals]
#plt.hist(obsPerPerson,bins=100)  

## 2.2 calculate the percentage of the valid macs in all macs of raw data

In [ ]:
#check the start time and end time of the ts_send
minTime=datetime.datetime.utcfromtimestamp(min(allTimes)/1000).strftime('%Y%m%d %H:%M:%S')
maxTime=datetime.datetime.utcfromtimestamp(max(allTimes)/1000).strftime('%Y%m%d %H:%M:%S')
#calculate the valid macs in the raw data
numValidMacsRaw=sum([individuals[ind]['valid'] for ind in individuals]) #number of the valid macs in raw data
numValidObs=sum([individuals[ind]['valid']*len(individuals[ind]['observations'])for ind in individuals])
numObs=sum([len(individuals[ind]['observations'])for ind in individuals])
pcentValid=100*numValidObs/numObs